In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
import pandas as pd

In [22]:
def RMSE(x,y):
  return np.sqrt(mean_squared_error(x,y))

In [49]:
set = pd.read_csv("waiting_times_train.csv")
set.describe()
set.head()

,DATETIME,ENTITY_DESCRIPTION_SHORT,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW,WAIT_TIME_IN_2H
0,2022-02-05 11:45:00,Water Ride,247.00,0,20,NaN,NaN,NaN,30.0
1,2019-02-24 10:45:00,Water Ride,247.00,0,30,375.0,NaN,495.0,25.0
2,2021-07-17 15:45:00,Pirate Ship,280.50,0,35,NaN,NaN,NaN,35.0
3,2022-04-03 19:45:00,Pirate Ship,230.35,0,15,-135.0,NaN,195.0,10.0
4,2021-10-20 10:30:00,Pirate Ship,153.00,0,15,NaN,NaN,NaN,10.0


In [50]:
meteo = pd.read_csv("weather_data.csv")
train_set = set.merge(meteo, on= "DATETIME", how = "left")


In [ ]:

null_rows = ["TIME_TO_PARADE_1","TIME_TO_PARADE_2","TIME_TO_NIGHT_SHOW"]

for j in null_rows:
    train_set[j] = train_set[j].fillna(600.0)

train_set["ENTITY_DESCRIPTION_SHORT"] = train_set["ENTITY_DESCRIPTION_SHORT"].replace({
    "Water Ride" : 0,
    "Pirate Ship": 0.5,
    "Flying Coaster": 1
})

train_set['DATETIME'] = pd.to_datetime(train_set['DATETIME'])
train_set['ANNEE'] = train_set['DATETIME'].dt.year
train_set['ANNEE'] = train_set['DATETIME'].dt.month
train_set['MINUTES_JOUR'] = train_set['DATETIME'].dt.hour * 60 + train_set['DATETIME'].dt.minute



train_set.head()

,DATETIME,ENTITY_DESCRIPTION_SHORT,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW,WAIT_TIME_IN_2H,temp,...,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,ANNEE,MOIS,JOUR,HEURE
0,2022-02-05 11:45:00,0.0,247.00,0,20,600.0,600.0,600.0,30.0,6.0175,...,1026.75,93.00,3.1500,0.383750,0.139065,64.0,2022,2,5,11
1,2019-02-24 10:45:00,0.0,247.00,0,30,375.0,600.0,495.0,25.0,7.9200,...,1035.75,41.75,3.0025,0.162423,0.347362,0.0,2019,2,24,10
2,2021-07-17 15:45:00,0.5,280.50,0,35,600.0,600.0,600.0,35.0,21.7700,...,1025.00,78.00,4.5325,0.243329,0.184573,100.0,2021,7,17,15
3,2022-04-03 19:45:00,0.5,230.35,0,15,-135.0,600.0,195.0,10.0,6.8650,...,1023.00,45.50,2.3425,0.242228,0.461012,86.5,2022,4,3,19
4,2021-10-20 10:30:00,0.5,153.00,0,15,600.0,600.0,600.0,10.0,15.1400,...,1010.50,88.50,6.8850,1.000000,0.181307,79.5,2021,10,20,10


In [52]:
Y = train_set['WAIT_TIME_IN_2H']
print(Y)
categories = train_set.columns.tolist()
print(categories)

0        30.0
1        25.0
2        35.0
3        10.0
4        10.0
         ... 
37013    10.0
37014    20.0
37015    10.0
37016    45.0
37017    20.0
Name: WAIT_TIME_IN_2H, Length: 37018, dtype: float64
['DATETIME', 'ENTITY_DESCRIPTION_SHORT', 'ADJUST_CAPACITY', 'DOWNTIME', 'CURRENT_WAIT_TIME', 'TIME_TO_PARADE_1', 'TIME_TO_PARADE_2', 'TIME_TO_NIGHT_SHOW', 'WAIT_TIME_IN_2H', 'temp', 'dew_point', 'feels_like', 'pressure', 'humidity', 'wind_speed', 'rain_1h', 'snow_1h', 'clouds_all']


In [53]:
def poly_fit(X, Y, deg):
            
    #prend X et renvoit un tableau avec les X, X**2 ect jusqu'à X**deg
    X_poly = PolynomialFeatures(degree=deg).fit_transform(X)
    
    lin_reg = LinearRegression()
    lin_reg.fit(X_poly,Y)
    return lin_reg

#pour appliquer la regression sur de nouvelles valeures
def poly_apply(lin_reg, degree,X):
    
    
    X_poly = PolynomialFeatures(degree=degree).fit_transform(X)
    return lin_reg.predict(X_poly)


In [54]:
#calcul du rmse + affichage des prédictions à coté des vraies valeurs
deg = 5
X_train = train_set[['DATETIME', 'ENTITY_DESCRIPTION_SHORT', 'ADJUST_CAPACITY', 'DOWNTIME', 'CURRENT_WAIT_TIME', 'TIME_TO_PARADE_1', 'TIME_TO_PARADE_2', 'TIME_TO_NIGHT_SHOW','temp', 'dew_point', 'feels_like', 'pressure', 'humidity', 'wind_speed', 'rain_1h', 'snow_1h', 'clouds_all']]
lin_reg = poly_fit(X_train,Y, deg = deg)

RMSE_train = RMSE(poly_apply(lin_reg,deg,X_train),Y)

plt.scatter(train_set['DATETIME'], Y, color='red', label='vraie valeure')
plt.scatter(train_set['DATETIME'], poly_apply(lin_reg,deg,X_train), color='blue', label='prédictions')#TODO
plt.xlabel('Date')
plt.ylabel('Temps attente dans deux heures')
plt.show()

ValueError: could not convert string to float: '2022-02-05 11:45:00'

In [ ]:
# Evaluate RMSE for polynomial degrees from 1 to 8
degrees = range(1, 9)  # Define the range of polynomial degrees to evaluate
RMSE_train_list = []  # List to store RMSE for training data
RMSE_test_list = []   # List to store RMSE for test data

# Loop through each degree, fit the model, and calculate RMSE
for deg in degrees:
    # Fit the polynomial regression model on the train with the current degree

    #TODO
    
    lin_reg = poly_fit(X_train, Y_train,deg, verbose = True)
    
    RMSE_train = RMSE(poly_apply(lin_reg, deg, X_train), Y_train)
    RMSE_test = RMSE(poly_apply(lin_reg, deg, X_test),Y_test)


    RMSE_train_list.append(RMSE_train)
    RMSE_test_list.append(RMSE_test)

    print(f"Degree = {deg}, RMSE_train = {RMSE_train:.3f}, RMSE_test = {RMSE_test:.3f}")

# Plot RMSE for training and test sets across different polynomial degrees
plt.plot(degrees, RMSE_train_list, label='Train RMSE', marker='o')
plt.plot(degrees, RMSE_test_list , label='Test RMSE', marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('RMSE')
plt.title('RMSE for Training and Test Sets')
plt.legend()
plt.show()